In [4]:
import sys, os
sys.path.append('/content/drive/MyDrive/dottorato 2022/XAI Fosca Giannotti/XAI-Lib')

In [5]:
# !pip install nam>=0.0.3

# Init NAMs with defaults values
[NAMs github repo](https://github.com/AmrMKayid/nam)

## NAMs automatic train, valid, test split

Synthetic dataset

In [ ]:
from xailib.explainers.nam_explainer_tab import NamTransparentByDesign, NamExplanation
import csv
import numpy as np
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
import torch


np.random.seed(0)

bs = 2  #This is the square root of the number of betas. Square root for plotting convenience
obs = 10000  #Number of obs
const = 5  #Constant a in y=a+beta*x+e
badd = 1  #Multiplies the normally distributed random betas by a constant
eadd = 1.7

#Creating the real underlying data
X = np.random.normal(size=[obs, bs**2])
errors = eadd * np.random.normal(size=[obs])
beta = badd * np.random.normal(size=[bs**2])
y = (const + np.matmul(np.sin(5 * X), beta) + errors)

In [ ]:
# additional_custom_config_params = dict(regression=False, seed=42) # see git repo for more
# model = NamTransparentByDesign(X, y, iris.feature_names, 'specie', **additional_custom_config_params)
model = NamTransparentByDesign(X=X, y=y, feature_names=["x1", "x2", "x3", "x4"], target_name='y')

In [ ]:
model.fit()

In [ ]:
model.test()

In [ ]:

item = torch.tensor([[-2.55298982,  0.6536186 ,  0.8644362 , -0.74216502],[ 7.45472240e-01,  3.47939136e-01, -2.59534011e-01,
    -4.23024293e-04], [-1.55298982,  0.2336186 ,  0.5544362 , -0.74216502]])
pred = model.predict(item)
display(pred)

### Local features importance

In [ ]:
exp = model.explain(item)
exp.getFeaturesImportance()

In [ ]:
exp.getFeaturesImportance().sum(1) 

In [ ]:
exp.getFeaturesImportance().sum(1) + exp.bias == pred

### Additional global explanation with shape functions

In [ ]:
model.explain_global()

## Manual train, valid, test split

In [ ]:
from nam.data import load_gallup_data, load_housing_data, load_sklearn_housing_data, load_breast_data
from nam.config import defaults
config = defaults()
nam_dataset = load_sklearn_housing_data(config)

In [ ]:
X = nam_dataset.features
y = nam_dataset.targets

In [ ]:
from torch.utils.data import DataLoader, Dataset

class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, item: int):
        return self.X[item], self.y[item]

In [ ]:
dataset = MyDataset(X,y)

In [ ]:
from torch.utils.data import random_split
batch_size = 1024

val_split, test_split = 0.1, 0.2

test_size = int(test_split * len(X))
val_size = int(val_split * (len(X) - test_size))
train_size = len(X) - val_size - test_size

train_subset, val_subset, test_subset = random_split(dataset, [train_size, val_size, test_size])

train_dl = DataLoader(train_subset, batch_size=batch_size, shuffle=True)

val_dl = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

test_dl = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

In [ ]:
model = NamTransparentByDesign(X=nam_dataset.features.numpy(), y=nam_dataset.targets, feature_names=nam_dataset.features_names, target_name=nam_dataset.targets_column)

In [ ]:
train_dataloaders = [(train_dl, val_dl)]

In [ ]:
model.fit(train_dataloaders)

In [ ]:
item = torch.tensor([[ 0.0761, -0.2157, -0.9236, -0.9621, -0.8656, -0.9977,  0.1307, -0.5757],
        [-0.0679,  1.0000, -0.8945, -0.9561, -0.9724, -0.9966,  0.1286, -0.5797],
        [-0.2906,  1.0000, -0.9295, -0.9561, -0.9689, -0.9970,  0.1286, -0.5817]])
pred = model.predict(item)
display(pred)

exp = model.explain(item)
display(exp.getFeaturesImportance())

exp.getFeaturesImportance().sum(1) + exp.bias == pred

In [ ]:
nam_dataset.features_names

In [ ]:
model.explain_global()